In [2]:
import boto3
import pandas as pd
from io import StringIO , BytesIO
from datetime import datetime , timedelta

In [3]:
arg_date = '2022-12-23'
src_format = '%Y-%m-%d'
src_bucket = 'xetra-1234'
trg_bucket = 'targetbucket-1234'
columns = ['ISIN',  'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume', 'NumberOfTrades']
trg_key = 'xetra_daiy_report_' + datetime.today().strftime('%Y-%m-%m_%H-%M-%S') + '.parquet'

In [4]:
arg_date_dt = datetime.strptime(arg_date ,src_format).date()-timedelta(days=1)

In [ ]:
bucket.Object()

In [5]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0],src_format).date() >= arg_date_dt]

In [5]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=",")
    return df

df_all = pd.concat([csv_to_df(obj.key) for obj in objects] , ignore_index=True)
    

In [6]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-12-22,08:00,36.090,36.090,35.830,35.860,912,7
1,DE000A1J5RX9,O2D,TELEFONICA DTLD HLDG NA,Common stock,EUR,2504492,2022-12-22,08:00,2.500,2.506,2.490,2.506,162400,27
2,DE000A1ML7J1,VNA,VONOVIA SE NA O.N.,Common stock,EUR,2504501,2022-12-22,08:00,45.410,45.410,45.210,45.240,47143,89
3,DE000A13SX22,HLE,HELLA GMBH+CO. KGAA O.N.,Common stock,EUR,2504580,2022-12-22,08:00,62.000,62.080,62.000,62.080,4591,2
4,DE000PAT1AG3,PAT,PATRIZIA AG NA ON,Common stock,EUR,2504817,2022-12-22,08:00,17.700,17.700,17.520,17.520,953,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352560,GB00BLD4ZP54,CLTC,COINSHARES DIG.SEC.OEND,ETN,EUR,6479084,2022-12-31,16:46,19.324,19.324,19.324,19.324,0,2
1352561,LU1923627332,RUSL,MUL-LYX.MSCI RUSSI.DIS.LS,ETF,EUR,5424594,2022-12-31,16:52,12.400,12.400,12.400,12.400,2645,2
1352562,US98956P1021,ZIM,ZIMMER BIOMET HLDGS DL-01,Common stock,EUR,4582018,2022-12-31,20:30,113.100,113.100,113.100,113.100,0,1
1352563,US9224171002,VEO,"VEECO INSTRUMENTS DL-,01",Common stock,EUR,6198311,2022-12-31,20:30,24.600,24.600,24.600,24.600,0,1


In [7]:
df_all = df_all.loc[:, columns]

In [8]:
df_all.dropna(inplace=True)

## Get the opening price per ISIN and day

In [9]:
df_all['opening_price'] = df_all.sort_values('Time').groupby(['ISIN','Date'])['StartPrice'].transform("first")

## Get closing price per ISIN and Day

In [10]:
df_all['closing_price'] = df_all.sort_values('Time').groupby(['ISIN','Date'])['StartPrice'].transform("last")

## Aggregrations

In [11]:
df_all = df_all.groupby(['ISIN','Date'], as_index=False).agg(opening_price_euro=('opening_price','min'),closing_price_euro=('closing_price','min'), minimun_price_euro=('MinPrice','min'),maximun_price_euro=('MaxPrice','max'), daily_traded_volume=('TradedVolume','sum'))

## Percentage Change previous closing price

In [12]:
df_all['previous_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_euro'].shift(1)

In [13]:
df_all['percentage_change_prev%'] = ((df_all['closing_price_euro']-df_all['previous_closing_price'])/df_all['previous_closing_price'])*100

In [14]:
df_all.drop(columns=['previous_closing_price'],inplace=True)

In [15]:
df_all = df_all.round(decimals=2)

In [16]:
df_all = df_all[df_all.Date >= arg_date]

## Write to s3

In [17]:
outbuffer =BytesIO()
df_all.to_parquet(outbuffer , index=False)
target_bucket = s3.Bucket(trg_bucket)
target_bucket.put_object(Body = outbuffer.getvalue() , Key = trg_key)

s3.Object(bucket_name='targetbucket-1234', key='xetra_daiy_report_2023-04-04_02-47-29.parquet')

## checking

In [18]:
for obj in target_bucket.objects.all():
    print(obj.key)

xetra_daiy_report2023-04-04_02-07-12.parquet
xetra_daiy_report_2023-04-04_02-13-42.parquet
xetra_daiy_report_2023-04-04_02-47-29.parquet


In [22]:
prq_obj = target_bucket.Object(key='xetra_daiy_report_2023-04-04_02-47-29.parquet').get().get('Body').read()

In [23]:
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [24]:
df_report

,ISIN,Date,opening_price_euro,closing_price_euro,minimun_price_euro,maximun_price_euro,daily_traded_volume,percentage_change_prev%
0,AT000000STR1,2022-12-23,37.00,37.95,37.00,37.95,396,2.99
1,AT000000STR1,2022-12-24,36.05,36.10,35.65,37.05,1838,-4.87
2,AT000000STR1,2022-12-25,36.10,37.70,36.10,37.70,2864,4.43
3,AT000000STR1,2022-12-26,36.10,37.70,36.10,37.70,2864,0.00
4,AT000000STR1,2022-12-27,36.10,37.70,36.10,37.70,2864,0.00
...,...,...,...,...,...,...,...,...
28846,XS2434891219,2022-12-27,3.44,3.50,3.44,3.50,0,0.00
28847,XS2434891219,2022-12-28,3.44,3.66,3.42,3.66,0,4.53
28848,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,0.00
28849,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.00
